In [1]:

%load_ext autoreload
%autoreload 2
import copy
from graph_algo.adt import LinkedGraph as LG, MatrixGraph as MG, Vertex, Edge, MapGraph
import  math

In [2]:
def make_graph(string, max_vertices):
    g = MapGraph()
    vertices = [Vertex(i) for i in range(max_vertices + 1)]
    for uv in string.split(" "):
        u, v = map(lambda x:int(x), uv.split("-"))
        g.insert_edge(Edge(vertices[u], vertices[v]))
    return g

In [3]:
def dfs(g, root=None, path=None):
    """Depth first search"""
    path = [] if path is None else path
    if root is None:
        for _ in g.vertices:
            root = _
            root.visited = True
            break

    for vertex in g.adjacent(root):
        edge = g.get_edge(root, vertex)
        if not hasattr(vertex, "visited"):
            vertex.visited = True
            path.append(edge)
            g.print_edge(edge)
            dfs(g, vertex, path)

    return path

dfs(make_graph("0-2 0-5 0-7 1-7 2-6 3-4 3-5 4-5 4-6 4-7",7))

0-2
2-6
4-6
3-4
3-5
4-7
1-7


[Edge(Vertex(value=0), Vertex(value=2)),
 Edge(Vertex(value=2), Vertex(value=6)),
 Edge(Vertex(value=4), Vertex(value=6)),
 Edge(Vertex(value=3), Vertex(value=4)),
 Edge(Vertex(value=3), Vertex(value=5)),
 Edge(Vertex(value=4), Vertex(value=7)),
 Edge(Vertex(value=1), Vertex(value=7))]

In [4]:
from data_structures import LinkedQueue
def bfs(g, root=None, path=None):
    """Depth first search"""
    path = [] if path is None else path
    if root is None:
        for _ in g.vertices:
            root = _
            root.visited = True
            path.append(root)
            break
    q = LinkedQueue()
    q.enqueue(root)
    while not q.is_empty():
        vertex = q.dequeue()
        for adj in g.adjacent(vertex):
            if not hasattr(adj, "visited"):
                q.enqueue(adj)
                adj.visited = True
                path.append(adj)
    return path

bfs(make_graph("0-2 0-5 0-7 1-7 2-6 3-4 3-5 4-5 4-6 4-7",7))

[Vertex(value=0),
 Vertex(value=2),
 Vertex(value=5),
 Vertex(value=7),
 Vertex(value=6),
 Vertex(value=3),
 Vertex(value=4),
 Vertex(value=1)]

In [116]:
from data_structures import LinkedStack
def cycle_detection(g):
    for edge in g:
        edge.is_traversed = False
    for vertex in g.vertices:
        vertex.is_visited = False
        vertex.parent = None
    stack = LinkedStack()
    for vertex in g.vertices:
        if not vertex.is_visited:
            stack.push(vertex)
            vertex.is_visited = True
        while not stack.is_empty():
            vertex = stack.pop()
            for adj in g.adjacent(vertex):
                if not adj.is_visited:
                    adj.is_visited = True
                    adj.parent = vertex
                    stack.push(adj)
                elif adj.is_visited and adj.parent and vertex != adj.parent:
                    return True

    return False



cycle_detection(make_graph("0-2 0-5 0-7 1-7 2-6 "
                           "3-4 3-5 4-5 4-6 4-7",7))
# cycle_detection(make_graph("0-1 0-2 2-3 0-4 4-5 4-6 5-6",6))

True

In [114]:
def find_cycle(g):
    for vertex in g.vertices:
        vertex.is_visited = False
        vertex.parent = None
    path = []
    stack = LinkedStack()
    for vertex in g.vertices:
        if not vertex.is_visited:
            stack.push(vertex)
            vertex.is_visited = True
        while not stack.is_empty():
            vertex = stack.pop()
            for adj in g.adjacent(vertex):
                if not adj.is_visited:
                    adj.is_visited = True
                    adj.parent = vertex
                    stack.push(adj)
                if adj.is_visited  and adj.parent and\
                        vertex != adj.parent:
                    curr_vertex =  vertex
                    path.append(curr_vertex)
                    while  curr_vertex != adj:
                        if curr_vertex.parent is None:
                            break
                        curr_vertex = curr_vertex.parent
                        path.append(curr_vertex)
                    if curr_vertex != adj:
                        sub_path = []
                        root , curr_vertex = curr_vertex, adj
                        while curr_vertex != root:
                            sub_path.append(curr_vertex)
                            curr_vertex = curr_vertex.parent
                        path = path + list(reversed(sub_path))

                    return path
    return  False

find_cycle(make_graph("0-2 0-5 0-7 1-7 2-6 "
                           "3-4 3-5 4-5 4-6 4-7",7))

[Vertex(value=4), Vertex(value=7), Vertex(value=0), Vertex(value=5)]

In [113]:
find_cycle(make_graph("0-3 0-2 0-1 2-6 2-5 2-4 4-1 1-7 2-7 7-8",8))


Vertex(value=7) Vertex(value=2) Vertex(value=0) Vertex(value=1)
Vertex(value=0) Vertex(value=1)
Vertex(value=0) Vertex(value=0)  D 
[Vertex(value=1)]


[Vertex(value=7), Vertex(value=2), Vertex(value=0), Vertex(value=1)]

In [119]:
# ame problem Two-colorability, bipartiteness, odd cycle
def is_bipartite(g):
    class Color:
        BLACK = 0
        WHITE = 1

    stack = LinkedStack()
    for vertex in g.vertices:
        vertex.is_visited = False
        vertex.color = None

    for vertex in g.vertices:
        if not vertex.is_visited:
            vertex.color = Color.BLACK
            stack.push(vertex)

        while not stack.is_empty():
            vertex = stack.pop()

            for adj in g.adjacent(vertex):
                if not adj.is_visited:
                    adj.color = Color.BLACK if vertex.color == Color.WHITE else Color.WHITE
                    adj.is_visited = True
                    stack.push(adj)
                elif adj.is_visited and vertex.color == adj.color:
                    return False
    return True



In [238]:
class C:
    count = 0
def get_bridges(g, vertex=None, res=None):
    if vertex is None:
        for i, v in enumerate(g.vertices):
            v.is_visited = False
            v.tin = -1
            v.low = -1
            v.parent=None
            # v.tin = float("inf")
            # v.low = float("inf")
            if i == 0:
                vertex = v
    vertex.is_visited = True
    vertex.tin =  C.count
    vertex.low = C.count
    C.count += 1
    for adj in g.adjacent(vertex):
        edge = g.get_edge(vertex, adj)
        if not adj.is_visited:
            # print("not visited")\
            adj.parent = vertex
            get_bridges(g, adj)
            # print("---")
            # print("return to", vertex, adj, adj.low )
            vertex.low = min(vertex.low, adj.low)
            print("-"*10)
            print(vertex, f"low: ",  vertex.low, f"tin: ", vertex.tin)
            print(adj, f"low: ", adj.low, f"tin: ", adj.tin)
            print("-"*10)
            if adj.low > vertex.tin:
                print(edge)
        else:
            if vertex.parent and vertex.parent != adj:
                vertex.low = min(vertex.low, adj.tin)

get_bridges(make_graph("0-1 0-5 0-6 1-2 2-6 3-4 3-5 4-5 4-9 4-11 "
           "6-7 7-8 7-10 8-10 9-11 11-12" , 12))
# get_bridges(make_graph("3-7 1-4 7-8 0-5 5-2 3-8 2-9 0-6 4-9 2-6 6-4", 9))

----------
Vertex(value=8) low:  4 tin:  5
Vertex(value=10) low:  4 tin:  6
----------
----------
Vertex(value=7) low:  4 tin:  4
Vertex(value=8) low:  4 tin:  5
----------
----------
Vertex(value=6) low:  0 tin:  3
Vertex(value=7) low:  4 tin:  4
----------
Edge(Vertex(value=6), Vertex(value=7))
----------
Vertex(value=2) low:  0 tin:  2
Vertex(value=6) low:  0 tin:  3
----------
----------
Vertex(value=1) low:  0 tin:  1
Vertex(value=2) low:  0 tin:  2
----------
----------
Vertex(value=0) low:  0 tin:  0
Vertex(value=1) low:  0 tin:  1
----------
----------
Vertex(value=11) low:  9 tin:  11
Vertex(value=12) low:  12 tin:  12
----------
Edge(Vertex(value=11), Vertex(value=12))
----------
Vertex(value=9) low:  9 tin:  10
Vertex(value=11) low:  9 tin:  11
----------
----------
Vertex(value=4) low:  7 tin:  9
Vertex(value=9) low:  9 tin:  10
----------
----------
Vertex(value=3) low:  7 tin:  8
Vertex(value=4) low:  7 tin:  9
----------
----------
Vertex(value=5) low:  7 tin:  7
Vertex(

In [242]:
class C:
    root = None
    count = 0
def get_articulation_point(g, vertex=None):
    """
    https://cp-algorithms.com/graph/cutpoints.html
    Let's say we are in the DFS, looking through the edges starting from vertex v≠root. If the current edge (v,to) is such that none of the vertices to or its descendants in the DFS traversal tree has a back-edge to any of ancestors of v, then v is an articulation point. Otherwise, v is not an articulation point.

Let's consider the remaining case of v=root. This vertex will be the point of articulation if and only if this vertex has more than one child in the DFS tree.
    :param g:
    :param vertex:
    :return:
    """
    if vertex is None:
        for i, v in enumerate(g.vertices):
            v.is_visited = False
            v.tin = -1
            v.low = -1
            v.parent=None
            # v.tin = float("inf")
            # v.low = float("inf")
            if i == 0:
                vertex = v
    if C.root is None:
        C.root = vertex
    vertex.is_visited = True
    vertex.tin =  C.count
    vertex.low = C.count
    children = 0
    C.count += 1
    for adj in g.adjacent(vertex):
        edge = g.get_edge(vertex, adj)
        if not adj.is_visited:
            # print("not visited")\
            adj.parent = vertex
            get_bridges(g, adj)
            # print("---")
            # print("return to", vertex, adj, adj.low )
            vertex.low = min(vertex.low, adj.low)
            print("-"*10)
            print(vertex, f"low: ",  vertex.low, f"tin: ", vertex.tin)
            print(adj, f"low: ", adj.low, f"tin: ", adj.tin)
            print("-"*10)
            if adj.low >= vertex.tin:
                print("Articulation point: ", vertex)

            children += 1
        else:
            if vertex.parent and vertex.parent != adj:
                vertex.low = min(vertex.low, adj.tin)
    if children > 1 and C.root == vertex:
        print("Articulation point: ", vertex)
get_articulation_point(make_graph("0-1 1-2 0-2 1-3 1-4 3-4", 4))
# get_articulation_point(make_graph("0-1 0-5 0-6 1-2 2-6 3-4 3-5 4-5 4-9 4-11 "
#            "6-7 7-8 7-10 8-10 9-11 11-12" , 12))


----------
Vertex(value=1) low:  0 tin:  1
Vertex(value=2) low:  0 tin:  2
----------
----------
Vertex(value=3) low:  1 tin:  3
Vertex(value=4) low:  1 tin:  4
----------
----------
Vertex(value=1) low:  0 tin:  1
Vertex(value=3) low:  1 tin:  3
----------
----------
Vertex(value=0) low:  0 tin:  0
Vertex(value=1) low:  0 tin:  1
----------
Articulation point:  Vertex(value=0)


In [ ]:
"""
Exercises: 3-7 1-4 7-8 0-5 5-2 3-8 2-9 0-6 4-9 2-6 6-4
"""




In [167]:
def find_shortest_path(g, start, end):
    q = LinkedQueue()
    q.enqueue(start)
    for vertex in g.vertices:
        vertex.distance = float("inf")
        vertex.is_visited = False
    start.is_visited = True
    start.distance = 0
    while not q.is_empty():
        vertex = q.dequeue()
        for adj in g.adjacent(vertex):
            if not adj.is_visited:
                adj.is_visited = True
                q.enqueue(adj)
                adj.distance = min(vertex.distance + 1, adj.distance)
    return end.distance



_g = make_graph("0-2 0-5 0-7 1-7 2-6 "
                           "3-4 3-5 4-5 4-6 4-7 1-8 8-9 9-0",9)
vertices = list(_g.vertices)

for i in range(1,10):
    print(i, find_shortest_path(_g, vertices[0], vertices[i]))

1 2
2 1
3 2
4 2
5 1
6 2
7 1
8 2
9 1
